In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="courtlin-udacity",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano")
exp = Experiment(workspace=ws, name="courtlin-udacity")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: courtlin-udacity
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano


In [6]:
from azureml.core import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

datastore = ws.get_default_datastore()

    
cpu_cluster_name = "udacityproj1"
if not cpu_cluster_name in ws.compute_targets :
    print('creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5, vm_priority='lowpriority',min_nodes=0, idle_seconds_before_scaledown=300)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True,min_node_count=None, timeout_in_minutes=20)

    # Show the result
    print(cpu_cluster.get_status().serialize())

cpu_cluster = ws.compute_targets[cpu_cluster_name]

creating a new compute target...
Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-07-29T05:29:06.762000+00:00', 'errors': None, 'creationTime': '2021-07-29T05:28:44.717728+00:00', 'modifiedTime': '2021-07-29T05:29:15.248655+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT300S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice

# ps = RandomParameterSampling( {
#         "learning_rate": normal(10, 3),
#         "keep_probability": uniform(0.05, 0.1),
#         "batch_size": choice(16, 32, 64, 128)
#     }
# )


ps = RandomParameterSampling(
    {
        "--C": choice(0.1,1.0,10.0),
        "--max_iter": choice(5,10,20)
    }
)


# Specify an Early Termination Policy
# policy = ### YOUR CODE HERE ###
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###


from azureml.core import ScriptRunConfig

# import os
# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

compute_target = ws.compute_targets['udacityproj1']

# from azureml.core import Environment
# sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

# src = ScriptRunConfig(source_directory=os.path.join('./'),
#                       script='train.py',
#                       arguments=['--kernel', 'linear', '--penalty', 1.0],
#                       compute_target=compute_target,
#                       environment=sklearn_env)

# est = SKLearn(source_directory=os.path.join('./'),
#                       entry_script='train.py',
#                       compute_target=cpu_cluster,
#              )

# Create a SKLearn estimator for use with train.py
# est = SKLearn(
#     source_directory=".",
#     compute_target=compute_target,
#     entry_script="train.py"
# )


# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=os.path.join('./'),
    compute_target=compute_target,
    entry_script="train.py"
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
                             estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=10)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "courtlin-udacity")
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True)

hyperdrive_run.wait_for_completion(show_output=True)

RunDetails(hyperdrive_run).show()



RunId: HD_f32066a8-f211-43e2-a351-3ff4c6a65db2
Web View: https://ml.azure.com/runs/HD_f32066a8-f211-43e2-a351-3ff4c6a65db2?wsid=/subscriptions/bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8/resourcegroups/udacity-nano/workspaces/courtlin-udacity&tid=3920f434-053b-4f53-aa54-b09a04e8f6dd

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-07-29T04:05:25.444528][API][INFO]Experiment created<END>\n""<START>[2021-07-29T04:05:25.808065][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2021-07-29T04:05:26.018453][GENERATOR][INFO]Successfully sampled '9' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_f32066a8-f211-43e2-a351-3ff4c6a65db2
Web View: https://ml.azure.com/runs/HD_f32066a8-f211-43e2-a351-3ff4c6a65db2?wsid=/subscriptions/bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8/resourcegroups/udacity-nano/workspaces/courtlin-udacity&tid=3920f434-053b-4f53-aa54-b09a04e8f6dd



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"



In [5]:
# from azureml.widgets import RunDetails
# RunDetails(hyperdrive_run).show()

In [6]:
import joblib
from azureml.core.model import Model

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

# model = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.pkl', model_framework=Model.Framework.SCIKITLEARN, model_framework_version='0.19.1')

### YOUR CODE HERE ###

# best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run)
print('Best Metrics:', best_run_metrics)
# print('\n Accuracy:', best_run_metrics['accuracy'])
# print('\n learning rate:',parameter_values[3])
# print('\n keep probability:',parameter_values[5])
# print('\n batch size:',parameter_values[7])

Best Run Id:  Run(Experiment: courtlin-udacity,
Id: HD_f32066a8-f211-43e2-a351-3ff4c6a65db2_3,
Type: azureml.scriptrun,
Status: Completed)
Best Metrics: {'Regularization Strength:': 1.0, 'Max iterations:': 20, 'accuracy': 0.9118361153262519}


In [7]:
import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)

best_run.get_file_names()
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/best_hyperdrive_model')

best_hyperdrive_model = best_run.register_model(
    model_name="best_hyperdrive_model",
    model_path="./outputs",
    tags=best_run.get_metrics()
)


# joblib.dump(value=best_run, filename="./outputs/best_hyperdrive_model.joblib")

# # print(fitted_model)
# # joblib.dump(best_run, "fitted_hyperdrive_model.joblib")

# # Save the trained model
# # os.makedirs('outputs', exist_ok=True)
# joblib.dump(value=best_run, filename='hyperdrive_model.pkl')

Run(Experiment: courtlin-udacity,
Id: HD_f32066a8-f211-43e2-a351-3ff4c6a65db2_3,
Type: azureml.scriptrun,
Status: Completed)


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

from azureml.core.dataset import Dataset

url_paths = [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
    ]

ds = TabularDatasetFactory.from_delimited_files(path=url_paths)
type(ds)

azureml.data.tabular_dataset.TabularDataset

In [9]:
# print(ds.to_pandas_dataframe())

In [10]:
from train import clean_data
from sklearn.model_selection import train_test_split

x, y = clean_data(ds)

#is this train/test split necessary with automl?
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=123)

# train_data = x.join(y)

# train_data.to_csv("data/train_data.csv", index=False)
# ds = ws.get_default_datastore()
# ds.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)
 
# Upload the training data as a tabular dataset for access during training on remote compute
# train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
# print(train_data.to_pandas_dataframe())
# clean_data = x
# clean_data['y'] = y 


# type(train_data)


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# automl_config = AutoMLConfig(
#     experiment_timeout_minutes=30,
#     task=,
#     primary_metric=,
#     training_data=,
#     label_column_name=,
#     n_cross_validations=)

automl_config = AutoMLConfig(
    experiment_timeout_hours=.5,
    task="classification",
    primary_metric="accuracy",
    training_data= ds,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_target,
    max_concurrent_iterations=5,
    max_cores_per_iteration=2,
    enable_onnx_compatible_models=True
)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)


Submitting remote run.
No run_configuration provided, running on udacityproj1 with default configuration
Running on remote compute: udacityproj1


Experiment,Id,Type,Status,Details Page,Docs Page
courtlin-udacity,AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
# best_run_metrics = best_automl_run.get_metrics() 
# for metric_name in best_run_metrics:
#      metric = best_run_metrics[metric_name]
#      print(metric_name, metric)


best_run, best_model = automl_run.get_output()

print("Best run:--------------------------------------------------")
print(best_run)
print("Best model:------------------------------------------------")
print(best_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Best run:--------------------------------------------------
Run(Experiment: courtlin-udacity,
Id: AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d_103,
Type: azureml.scriptrun,
Status: Completed)
Best model:------------------------------------------------
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=0.625, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2727272727272727, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909

In [14]:
best_model 

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mn...
), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=0.625, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2727272727272727, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [15]:
# Returns the metrics
print("Best run metrics :")
best_run.get_metrics()


Best run metrics :


{'precision_score_weighted': 0.9125786451564377,
 'AUC_weighted': 0.9496901959257948,
 'matthews_correlation': 0.5576671393914474,
 'precision_score_micro': 0.9177238239757208,
 'recall_score_micro': 0.9177238239757208,
 'average_precision_score_micro': 0.9821724365395423,
 'AUC_micro': 0.9813789274686204,
 'accuracy': 0.9177238239757208,
 'recall_score_macro': 0.7576691324860108,
 'f1_score_weighted': 0.9144972994549061,
 'average_precision_score_macro': 0.8298840018094473,
 'weighted_accuracy': 0.9574756787950088,
 'recall_score_weighted': 0.9177238239757208,
 'precision_score_macro': 0.8018276411085768,
 'f1_score_macro': 0.7771886791090374,
 'f1_score_micro': 0.9177238239757208,
 'balanced_accuracy': 0.7576691324860108,
 'AUC_macro': 0.9496901959257948,
 'average_precision_score_weighted': 0.9568546104647018,
 'log_loss': 0.1740862181951805,
 'norm_macro_recall': 0.5153382649720215,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d_

In [16]:
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :")
best_run.get_details()

Best run details :


{'runId': 'AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d_103',
 'target': 'udacityproj1',
 'status': 'Completed',
 'startTimeUtc': '2021-07-29T04:51:43.516619Z',
 'endTimeUtc': '2021-07-29T04:53:21.875162Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'courtlin-udacity\',\'compute_target\':\'udacityproj1\',\'subscription_id\':\'bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d_103","experiment_name":"courtlin-udacity","workspace_name":"courtlin-udacity","subscription_id":"bd6c48f0-b2f5-4fd8-b4de-3351b1

In [17]:
best_run_metrics = automl_run.get_metrics() 
for metric_name in best_run_metrics:
     metric = best_run_metrics[metric_name]
     print(metric_name, metric)

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetBalancing', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Performing class balancing sweeping', 'Generating individually featurized CV splits.', 'Beginning model selection.']
recall_score_weighted 0.9177238239757208
AUC_micro 0.9813789274686204
recall_score_micro 0.9177238239757208
f1_score_macro 0.7771886791090374
f1_score_micro 0.9177238239757208
average_precision_score_weighted 0.9568546104647018
precision_score_micro 0.9177238239757208
log_loss 0.1740862181951805
accuracy 0.9177238239757208
norm_macro_recall 0.5153382649720215
AUC_macro 0.9496901959257948
weighted_accuracy 0.9574756787950088
balanced_accuracy 0.7576691324860108
f1_score

In [18]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [19]:
import joblib
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, "fitted_automl_model.joblib")

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Run(Experiment: courtlin-udacity,
Id: AutoML_c4580b80-2401-4a12-8df3-0dccd3a4570d_103,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=1.4583333333333335, reg_lambda=0.625, subsample=0.6, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.2727272727272727, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


['fitted_automl_model.joblib']

In [ ]:
# fitted_model.steps

In [24]:
from pprint import pprint
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()
print_model(fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': False,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': True,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/courtlin1/code/Users/courtlin/COURTLIN_Optimizing_a_Pipeline_in_Azure-Starter_Files-master'}

prefittedsoftvotingclassifier
{'estimators': ['88', '69', '94', '30', '39', '97', '53', '57'],
 'weights': [0.2727272727272727,
             0.09090909090909091,
             0.18181818181818182,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091]}

88 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

88 - x

In [20]:
cpu_cluster.get_status()

In [21]:
# Delete compute cluster

cpu_cluster.delete()

In [22]:
cpu_cluster.get_status()

Current provisioning state of AmlCompute is "Deleting"

